In [222]:
from IPython.display import display, Markdown, Latex, HTML
import numpy as np
from operator import itemgetter

def dicts_list_as_table(source_dict):
    keys = list(key for key in source_dict[0].keys())
    keys = r'|'.join(keys)
    tbl = [r'|' + keys +r'|',]
    tbl.append(r'|' + r'---|'.join('' for key in source_dict[0].keys()) + r'|')

    for item in source_dict:
        line = list(str(value) for value in item.values())
        tbl.append(r'| %s |' % r'|'.join(line))

    display(Markdown('\n'.join(tbl)))

# Лабораторна робота №1

## Вихідні дані

In [252]:
source_data = [{
 'name': 'Kingston SSDNow KC400',
 'size': 128,
 'read_speed': 550,
 'write_speed': 450,
 'price': 0.1,
 'lifetime': 1
},
{
 'name': 'Kingston HyperX Fury',
 'size': 120,
 'read_speed': 500,
 'write_speed': 500,
 'price': 0.5,
 'lifetime': 1
},
{
'name': 'Kingston SSDNow V300',
 'size': 120,
 'read_speed': 450,
 'write_speed': 450,
 'price': 0.4,
 'lifetime': 1
},]

dicts_list_as_table(source_data)

|lifetime|read_speed|write_speed|price|size|name|
|---|---|---|---|---||
| 1|550|450|0.1|128|Kingston SSDNow KC400 |
| 1|500|500|0.5|120|Kingston HyperX Fury |
| 1|450|450|0.4|120|Kingston SSDNow V300 |

## Нормовані показники

In [253]:
standartized_values = []

# Отримуємо максимальні значення
max_size = max([item['size'] for item in source_data])
max_read_speed = max([item['read_speed'] for item in source_data])
max_write_speed = max([item['write_speed'] for item in source_data])
max_price = max([item['price'] for item in source_data])
max_lifetime = max([item['lifetime'] for item in source_data])

for item in source_data:
    standartized_values.append(
        {
            'name': item['name'],
            'size': round((item['size'] / max_size), 3),
            'read_speed': round((item['read_speed'] / max_read_speed),3),
            'write_speed': round((item['write_speed'] / max_write_speed), 3),
            'price': round((item['price'] / max_price), 3),
            'lifetime': round((item['lifetime'] / max_lifetime), 3)
        }
    )

dicts_list_as_table(standartized_values)


|lifetime|read_speed|write_speed|price|size|name|
|---|---|---|---|---||
| 1.0|1.0|0.9|0.2|1.0|Kingston SSDNow KC400 |
| 1.0|0.909|1.0|1.0|0.938|Kingston HyperX Fury |
| 1.0|0.818|0.9|0.8|0.938|Kingston SSDNow V300 |

In [116]:
# Друк "ручних розрахунків"

eq_count = 0

for item in standartized_values:
    _sum = 0
    formula = []
    i = 0
    for value in item.values():
        if type(value) is not str:
            _sum =+ value * coefficients[i]
            formula.append(r'%s \cdot %s' % (value, coefficients[i]))
            i =+1
    
    formula = r' + '.join(formula)
    eq_count =+ 1
    formula = r'\begin{equation} F_{%s} = %s = %s \end{equation}' % (eq_count, formula, _sum)
    
    display(HTML(r'<pre>' + formula + '</pre>'))
    display(Latex(formula))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Знаходження оптимального варіанту

In [254]:
coefficients = [0.05, 0.3, 0.3, -0.15, 0.2]
scores = []

def get_optimal():
    """Повертає оптимальний варіант"""
    return max(standartized_values, key=lambda x:x['score'])

def calculate_efficiency(coefficients=coefficients):
    """Рахуємо загальну ефективність"""
    for item in standartized_values:
        _sum = 0
        i = 0
        for value in item.values():
            if type(value) is not str:
                _sum =+ value * coefficients[i]
                i =+1
                
        item['score'] = _sum
#         item['coefficient'] = coefficients[i]
    
calculate_efficiency()
dicts_list_as_table(standartized_values)

max_score = get_optimal()

display(Markdown(r'__Оптимальний варіант__: %s' % max_score['name']))

|lifetime|read_speed|write_speed|score|price|size|name|
|---|---|---|---|---|---||
| 1.0|1.0|0.9|0.3|0.2|1.0|Kingston SSDNow KC400 |
| 1.0|0.909|1.0|0.2814|1.0|0.938|Kingston HyperX Fury |
| 1.0|0.818|0.9|0.2814|0.8|0.938|Kingston SSDNow V300 |

__Оптимальний варіант__: Kingston SSDNow KC400

# Лабораторна робота №2

In [272]:
coefficients = [0.05, 0.3, 0.3, -0.15, 0.2] # Коефіцієнти
step = 0.05                                 # Крок
coefficients_cnt = len(coefficients)        # Кількість коефіцієнтів

def abs_sum(lst):
    """Повертає суму елементів списку по модулю"""
    sum_ = 0
    for item in lst:
        sum_ = sum_ + float(abs(item))
    return(sum_)


def find_max(coefficients, i):
    results = []
    
    for delta in np.arange(step, 1, step):
        new_coeff = list(coefficients)
        
        # Додаємо Delta до поточного коефіцієнту
        new_coeff[i] = new_coeff[i] + delta
        
        for c in range(0, len(new_coeff)):
            if i != c:
                # Від усіх коефіцієнтів окрім поточного віднімаємо (Delta / кільк. коеф)
                new_coeff[c] =new_coeff[c] - delta / (coefficients_cnt - 1)
        
        # Отримуємо суму по модулю
        sum_abs =abs_sum(new_coeff)
        
        # Якщо сума по модулю менше одиниці додаємо її в список результатів
        if sum_abs <= 1:
            results.append([new_coeff[i] , sum_abs, new_coeff ])
    
    
    if results:
        # Шукаємо максимальне значення у спсику результатів
        calculate_efficiency(coefficients=max(results, key=itemgetter(0))[2])
        
        if get_optimal() == max_score:
            print('Максимальне значення %s: %s' % (i+1, max(results, key=itemgetter(0))[0]))
            dicts_list_as_table(standartized_values)
            print('Оптимальний варіант: %s' % get_optimal()['name'])
        else:
            calculate_efficiency()
            print('Максимальне значення %s: %s' % (i+1, coefficients[i]))
            dicts_list_as_table(standartized_values)
            print('Оптимальний варіант: %s' % get_optimal()['name'])
            
    else:
        calculate_efficiency()
        print('Максимальне значення %s: %s' % (i+1, coefficients[i]))
        dicts_list_as_table(standartized_values)
        print('Оптимальний варіант: %s' % get_optimal()['name'])
        

        
def find_min(coefficients, i):
    results = []
    
    for delta in np.arange(step, 1, step):
        new_coeff = list(coefficients)
        
        new_coeff[i] = new_coeff[i] - delta
        
        for c in range(0, len(new_coeff)):
            if i != c:
                new_coeff[c] =new_coeff[c] + delta / (coefficients_cnt - 1)
        
        sum_abs =abs_sum(new_coeff)
        
        if sum_abs <= 1:
            results.append([new_coeff[i] , sum_abs, new_coeff ])
        
    if results:
        # Шукаємо максимальне значення у спсику результатів
        
        calculate_efficiency(coefficients=min(results, key=itemgetter(0))[2])
        if get_optimal() == max_score:
            print('Мінімальне значення %s: %s' % (i+1, min(results, key=itemgetter(0))[0]))
            dicts_list_as_table(standartized_values)
            print('Оптимальний варіант: %s' % get_optimal()['name'])
            
        else:
            calculate_efficiency()
            print('Мінімальне значення %s: %s' % (i+1, coefficients[i]))
            dicts_list_as_table(standartized_values)
            print('Оптимальний варіант: %s' % get_optimal()['name'])
        
    else:
        print('Мінімальне значення %s: %s' % (i+1, coefficients[i]))
        dicts_list_as_table(standartized_values)
        print('Оптимальний варіант: %s' % get_optimal()['name'])



for i in range(0, coefficients_cnt):
    
    display(Markdown('#### Обрахунки для значення %s' % str(i+1)))
    find_max(coefficients, i)
    find_min(coefficients, i)
    print('\n')
    

#### Обрахунки для значення 1

Максимальне значення 1: 0.05


|lifetime|read_speed|write_speed|score|price|size|name|
|---|---|---|---|---|---||
| 1.0|1.0|0.9|0.3|0.2|1.0|Kingston SSDNow KC400 |
| 1.0|0.909|1.0|0.2814|1.0|0.938|Kingston HyperX Fury |
| 1.0|0.818|0.9|0.2814|0.8|0.938|Kingston SSDNow V300 |

Оптимальний варіант: Kingston SSDNow KC400
Мінімальне значення 1: 0.0


|lifetime|read_speed|write_speed|score|price|size|name|
|---|---|---|---|---|---||
| 1.0|1.0|0.9|0.3125|0.2|1.0|Kingston SSDNow KC400 |
| 1.0|0.909|1.0|0.293125|1.0|0.938|Kingston HyperX Fury |
| 1.0|0.818|0.9|0.293125|0.8|0.938|Kingston SSDNow V300 |

...

Максимальне значення 5: 0.2


|lifetime|read_speed|write_speed|score|price|size|name|
|---|---|---|---|---|---||
| 1.0|1.0|0.9|0.3|0.2|1.0|Kingston SSDNow KC400 |
| 1.0|0.909|1.0|0.2814|1.0|0.938|Kingston HyperX Fury |
| 1.0|0.818|0.9|0.2814|0.8|0.938|Kingston SSDNow V300 |

Оптимальний варіант: Kingston SSDNow KC400
Мінімальне значення 5: -0.05


|lifetime|read_speed|write_speed|score|price|size|name|
|---|---|---|---|---|---||
| 1.0|1.0|0.9|0.3625|0.2|1.0|Kingston SSDNow KC400 |
| 1.0|0.909|1.0|0.340025|1.0|0.938|Kingston HyperX Fury |
| 1.0|0.818|0.9|0.340025|0.8|0.938|Kingston SSDNow V300 |

Оптимальний варіант: Kingston SSDNow KC400


